<a href="https://colab.research.google.com/github/yuann3/YOLO-Research-Report/blob/main/yolo_research_biomed_blood_cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO for Biomedical Image Detection 2

we evaluate the model using Precision, Recall, F1-score, mAP@50, mAP@50-95, and FPS

since there is a lot of blood-cells dataset on roboflow with good support, part of the code is from ultralytics offical doc (but we know this is execuse cuz ddl is tmr and i havent write anything yet)

## Install YOLO11 via Ultralytics

In [ ]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

!pip install roboflow
!pip install matplotlib seaborn pandas numpy
!pip install scikit-learn
!pip install supervision

HOME = "/content"

## Dataset download

In [ ]:
!mkdir -p {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

In [ ]:
from roboflow import Roboflow

# who da fak will put put their api key here
# oh is me cuz at this stage i really dont care about it that much
rf = Roboflow(api_key="SHNYKA5a9WL7F1HrKRNh")
project = rf.workspace("team-roboflow").project("blood-cell-detection-1ekwu")
version = project.version(3)
dataset = version.download("yolov11")

ok lets make sure the this shit is working

In [ ]:
%cd {dataset.location}
!ls
!cat data.yaml

# Analysis the dataset

In [ ]:
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import numpy as np
from PIL import Image

SyntaxError: invalid syntax (<ipython-input-1-be933fffc6f5>, line 1)

In [ ]:
with open(f"{dataset.location}/data.yaml", 'r') as file:
    data_config = yaml.safe_load(file)

print("Classes:", data_config['names'])
print("Number of classes:", data_config['nc'])

# Analyze class distribution in training set
train_labels_dir = os.path.join(dataset.location, "train", "labels")
val_labels_dir = os.path.join(dataset.location, "valid", "labels")

def count_classes(labels_dir, class_count):
    for label_file in os.listdir(labels_dir):
        if label_file.endswith('.txt'):
            with open(os.path.join(labels_dir, label_file), 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 1:
                        class_id = int(parts[0])
                        class_count[class_id] += 1
    return class_count

class_count = [0] * data_config['nc']

class_count = count_classes(train_labels_dir, class_count)

df_classes = pd.DataFrame({
    'Class': data_config['names'],
    'Count': class_count
})

plt.figure(figsize=(12, 6))
sns.barplot(x='Class', y='Count', data=df_classes)
plt.title('Class Distribution in Training Set')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

train_images_dir = os.path.join(dataset.location, "train", "images")
image_sizes = []

for img_file in os.listdir(train_images_dir)[:100]:  # limit to 100 images for speed cuz im running out credit and i need my 4090 to play minecraft
    if img_file.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(train_images_dir, img_file)
        img = Image.open(img_path)
        width, height = img.size
        image_sizes.append((width, height))

widths, heights = zip(*image_sizes)

# i sill got 12 hours left, 2 more ddls, chill
plt.figure(figsize=(12, 6))
plt.scatter(widths, heights, alpha=0.5)
plt.xlabel('Width (pixels)')
plt.ylabel('Height (pixels)')
plt.title('Image Dimensions in Training Set')
plt.grid(True)
plt.show()

# calculate average image dimensions
avg_width = sum(widths) / len(widths)
avg_height = sum(heights) / len(heights)
print(f"Average image dimensions: {avg_width:.1f} x {avg_height:.1f} pixels")

NameError: name 'dataset' is not defined

# Model Training

In [ ]:
%cd {HOME}

# training config since the dataset is big so we just go epochs 50+ above
!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=50 imgsz=640 plots=True patience=15 save_period=10

# Model Evaluation

In [ ]:
%cd {HOME}

In [ ]:
# Run validation on the test set
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml split=test
```

In [ ]:
detailed evaluation

In [ ]:
```python
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_fscore_support
import time
from ultralytics import YOLO
from pathlib import Path
import glob
from PIL import Image
import supervision as sv

In [ ]:
# Load the trained model
model = YOLO(f"{HOME}/runs/detect/train/weights/best.pt")

In [ ]:
# Function to parse validation results
def parse_results(results_file):
    with open(results_file, 'r') as f:
        for line in f:
            if line.startswith('                 Class'):
                header_line = line.strip()
            if line.startswith('                   all'):
                metrics_line = line.strip()
                break

    headers = header_line.split()
    metrics = metrics_line.split()

    class_name = metrics[0]
    images = int(metrics[1])
    instances = int(metrics[2])
    precision = float(metrics[3])
    recall = float(metrics[4])
    mAP50 = float(metrics[5])
    mAP50_95 = float(metrics[6])

    return {
        'Class': class_name,
        'Images': images,
        'Instances': instances,
        'Precision': precision,
        'Recall': recall,
        'mAP50': mAP50,
        'mAP50-95': mAP50_95
    }

In [ ]:
# Get the path to validation results
val_results_file = sorted(Path(f"{HOME}/runs/detect").glob("val*/results.txt"))[-1]

In [ ]:
# Parse validation results
overall_metrics = parse_results(val_results_file)

In [ ]:
# Display overall metrics
print("\nOverall Metrics:")
print(f"Precision: {overall_metrics['Precision']:.4f}")
print(f"Recall: {overall_metrics['Recall']:.4f}")
print(f"mAP50: {overall_metrics['mAP50']:.4f}")
print(f"mAP50-95: {overall_metrics['mAP50-95']:.4f}")

In [ ]:
# Calculate F1 score
f1_score = 2 * (overall_metrics['Precision'] * overall_metrics['Recall']) / (overall_metrics['Precision'] + overall_metrics['Recall'])
print(f"F1 Score: {f1_score:.4f}")

In [ ]:
# FPS Measurement
def measure_fps(model, test_images, num_runs=100):
    # Warmup
    for _ in range(10):
        _ = model(test_images[0])

    # Measure inference time
    start_time = time.time()
    for i in range(min(num_runs, len(test_images))):
        _ = model(test_images[i])
    end_time = time.time()

    elapsed_time = end_time - start_time
    fps = num_runs / elapsed_time
    return fps

In [ ]:
# Load some test images for FPS measurement
test_images_path = os.path.join(dataset.location, "test", "images")
test_images = [os.path.join(test_images_path, img) for img in os.listdir(test_images_path)
               if img.endswith(('.jpg', '.jpeg', '.png'))][:100]  # Limit to 100 images

In [ ]:
# Measure FPS
fps = measure_fps(model, test_images)
print(f"\nInference Speed: {fps:.2f} FPS")

In [ ]:
# Create a metrics table
metrics_table = pd.DataFrame({
    'Metric': ['Precision', 'Recall', 'F1 Score', 'mAP50', 'mAP50-95', 'FPS'],
    'Value': [
        overall_metrics['Precision'],
        overall_metrics['Recall'],
        f1_score,
        overall_metrics['mAP50'],
        overall_metrics['mAP50-95'],
        fps
    ]
})

In [ ]:
print("\nMetrics Summary:")
print(metrics_table)

In [ ]:
# Save metrics to CSV for inclusion in the paper
metrics_table.to_csv(f"{HOME}/biomedical_detection_metrics.csv", index=False)
```

# Visualize Evaluation Results

In [ ]:
```python
# Create a bar plot of the main metrics
plt.figure(figsize=(12, 6))
sns.barplot(x='Metric', y='Value', data=metrics_table[metrics_table['Metric'] != 'FPS'])
plt.title('YOLOv11 Performance Metrics on Biomedical Dataset')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig(f"{HOME}/metrics_barplot.png")
plt.show()

In [ ]:
# Plot precision-recall curve (if data is available from validation)
try:
    pr_curve = pd.read_csv(f"{HOME}/runs/detect/train/results.csv")
    plt.figure(figsize=(10, 6))
    plt.plot(pr_curve['               recall'], pr_curve['             precision'], marker='o', linewidth=2)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.grid(True)
    plt.savefig(f"{HOME}/precision_recall_curve.png")
    plt.show()
except:
    print("Could not load precision-recall data from results.csv")

In [ ]:
# Visualize class-wise metrics from validation results
val_data = []
with open(val_results_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 7 and parts[0] != 'Class' and parts[0] != 'all':
            try:
                class_name = parts[0]
                precision = float(parts[3])
                recall = float(parts[4])
                mAP50 = float(parts[5])
                mAP50_95 = float(parts[6])
                val_data.append({
                    'Class': class_name,
                    'Precision': precision,
                    'Recall': recall,
                    'mAP50': mAP50,
                    'mAP50-95': mAP50_95
                })
            except:
                continue

In [ ]:
if val_data:
    df_val = pd.DataFrame(val_data)

    # Plot class-wise metrics
    plt.figure(figsize=(12, 8))
    df_melted = pd.melt(df_val, id_vars=['Class'], var_name='Metric', value_name='Value')
    sns.barplot(x='Class', y='Value', hue='Metric', data=df_melted)
    plt.title('Class-wise Performance Metrics')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.savefig(f"{HOME}/class_metrics.png")
    plt.show()
```

# Inference

In [ ]:
```python
%cd {HOME}

In [ ]:
# Run inference on test images
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
# Visualize some results
import glob
from IPython.display import Image as IPyImage, display

In [ ]:
latest_folder = max(glob.glob(f'{HOME}/runs/detect/predict*/'), key=os.path.getmtime)
print(f"Visualization results saved to: {latest_folder}")

In [ ]:
# Show first 5 predictions
for img in sorted(glob.glob(f'{latest_folder}/*.jpg'))[:5]:
    display(IPyImage(filename=img, width=600))
    print("\n")
```

# Advanced Analysis for Research Paper

In [ ]:
 some additional code to generate visualizations and analyses for our research paper:

In [ ]:
```python
# Confusion Matrix Analysis
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image as IPyImage, display

In [ ]:
# Display confusion matrix from training run
confusion_matrix_path = f"{HOME}/runs/detect/train/confusion_matrix.png"
if os.path.exists(confusion_matrix_path):
    display(IPyImage(filename=confusion_matrix_path, width=800))
    print("Confusion Matrix from Training")

In [ ]:
# Calculate and visualize additional metrics for the paper
if val_data:
    df_val = pd.DataFrame(val_data)

    # Calculate F1 scores for each class
    df_val['F1-Score'] = 2 * (df_val['Precision'] * df_val['Recall']) / (df_val['Precision'] + df_val['Recall'])

    # Radar chart for multi-dimensional performance metrics
    from matplotlib.path import Path
    from matplotlib.splines import Spline
    from matplotlib.patches import Circle, RegularPolygon
    from matplotlib.projections import register_projection
    from matplotlib.projections.polar import PolarAxes
    from matplotlib.transforms import Affine2D


   # not quite sure what im doing here, but it works
    def radar_factory(num_vars, frame='circle'):
        theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
        class RadarAxes(PolarAxes):
            name = 'radar'

            def __init__(self, *args, **kwargs):
                super().__init__(*args, **kwargs)
                self.set_theta_zero_location('N')

            def fill(self, *args, closed=True, **kwargs):
                return super().fill(self.theta, args[0], closed=closed, **kwargs)

            def plot(self, *args, **kwargs):
                return super().plot(self.theta, args[0], **kwargs)

            def set_varlabels(self, labels):
                self.set_thetagrids(np.degrees(self.theta), labels)

            def _gen_axes_patch(self):
                if frame == 'circle':
                    return Circle((0.5, 0.5), 0.5)
                elif frame == 'polygon':
                    return RegularPolygon((0.5, 0.5), num_vars, radius=0.5, edgecolor="k")
                else:
                    raise ValueError("Unknown value for 'frame': %s" % frame)

            def _gen_axes_spines(self):
                if frame == 'circle':
                    return super()._gen_axes_spines()
                elif frame == 'polygon':
                    spine_dict = {}
                    for i in range(num_vars):
                        spine = Spline(self.Axes.path, self._path_transform)
                        spine.set_transform(self.transAxes)
                        spine_dict[self.Axes.get_side(i)] = spine
                    return spine_dict
                else:
                    raise ValueError("Unknown value for 'frame': %s" % frame)

        register_projection(RadarAxes)
        return theta

    # Get top 5 classes by mAP50
    top_classes = df_val.sort_values('mAP50', ascending=False).head(5)

    # Prepare data for radar chart
    metrics = ['Precision', 'Recall', 'F1-Score', 'mAP50', 'mAP50-95']
    data = []
    for _, row in top_classes.iterrows():
        values = [row[metric] for metric in metrics]
        data.append(values)

    # Create radar chart
    theta = radar_factory(len(metrics), frame='polygon')
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='radar'))

    colors = ['b', 'g', 'r', 'c', 'm']
    for i, (class_data, color) in enumerate(zip(data, colors)):
        ax.plot(class_data, color=color)
        ax.fill(class_data, alpha=0.1, color=color)

    ax.set_varlabels(metrics)
    ax.set_ylim(0, 1)
    plt.legend(top_classes['Class'].values, loc='upper right')
    plt.title('Multi-dimensional Performance Analysis of Top 5 Classes')
    plt.savefig(f"{HOME}/radar_metrics.png", dpi=300, bbox_inches='tight')
    plt.show()

    # Create a heatmap of metrics by class
    plt.figure(figsize=(12, len(df_val) * 0.5))
    sns.heatmap(df_val[metrics].values,
                annot=True,
                fmt=".3f",
                cmap="YlGnBu",
                yticklabels=df_val['Class'],
                xticklabels=metrics,
                vmin=0, vmax=1)
    plt.title('Performance Metrics Heatmap by Class')
    plt.tight_layout()
    plt.savefig(f"{HOME}/metrics_heatmap.png", dpi=300, bbox_inches='tight')
    plt.show()
```

# Results Analysis and Interpretation

In [ ]:
```python

In [ ]:
print("\n\n---------- RESULTS SUMMARY ----------")
print(f"\nOverall Performance Metrics:")
print(f"Precision: {overall_metrics['Precision']:.4f}")
print(f"Recall: {overall_metrics['Recall']:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"mAP@50: {overall_metrics['mAP50']:.4f}")
print(f"mAP@50-95: {overall_metrics['mAP50-95']:.4f}")
print(f"Inference Speed: {fps:.2f} FPS")

In [ ]:
# Interpretation of results
print("\nInterpretation of Results:")
print("""
1. **Overall Performance**: The YOLOv11 model achieved [precision] precision, [recall] recall, and an F1 score of [f1_score], indicating [strong/moderate/weak] performance on the biomedical dataset.

2. **Detection Accuracy**: The mAP@50 of [mAP50] and mAP@50-95 of [mAP50-95] demonstrate that the model performs [well/acceptably/poorly] across different IoU thresholds, suggesting [robust/variable] detection capabilities.

3. **Real-time Capabilities**: With an inference speed of [fps] FPS, the model [is/is not] suitable for real-time applications in biomedical image analysis.

4. **Class-wise Performance**: Analysis of class-wise metrics reveals that [best_classes] showed the highest detection accuracy, while [worst_classes] had lower performance. This suggests that the model [is more effective at/struggles with] detecting certain types of biomedical objects.

5. **Comparison to Literature**: These results [exceed/are comparable to/fall short of] the current state-of-the-art in biomedical object detection, which typically reports mAP50 values between [literature_range].

6. **Practical Implications**: The evaluation metrics indicate that YOLOv11 [is/is not] a viable option for biomedical image analysis tasks, particularly for [specific applications].
""".replace('[precision]', f"{overall_metrics['Precision']:.4f}")
   .replace('[recall]', f"{overall_metrics['Recall']:.4f}")
   .replace('[f1_score]', f"{f1_score:.4f}")
   .replace('[mAP50]', f"{overall_metrics['mAP50']:.4f}")
   .replace('[mAP50-95]', f"{overall_metrics['mAP50-95']:.4f}")
   .replace('[fps]', f"{fps:.2f}")
)

In [ ]:
# Generate JSON results for programmatic use
results_json = {
    "model": "YOLOv11s",
    "dataset": "Blood Cell Detection",
    "metrics": {
        "precision": overall_metrics['Precision'],
        "recall": overall_metrics['Recall'],
        "f1_score": f1_score,
        "mAP50": overall_metrics['mAP50'],
        "mAP50-95": overall_metrics['mAP50-95'],
        "fps": fps
    },
    "training_details": {
        "epochs": 50,
        "image_size": 640,
        "optimizer": "Adam",
        "batch_size": 16
    }
}

In [ ]:
import json
with open(f"{HOME}/results_summary.json", 'w') as f:
    json.dump(results_json, f, indent=4)

In [ ]:
print(f"\nResults summary saved to {HOME}/results_summary.json")
```

# Comparison with Different Versions of the Dataset

In [ ]:
```python
# Function to train and evaluate on different dataset versions
def train_and_evaluate(version_number):
    print(f"\n\n---------- EVALUATING DATASET VERSION {version_number} ----------")

    # Download dataset
    %cd {HOME}/datasets

    from roboflow import Roboflow
    rf = Roboflow(api_key="SHNYKA5a9WL7F1HrKRNh")
    project = rf.workspace("team-roboflow").project("blood-cell-detection-1ekwu")
    version = project.version(version_number)
    dataset = version.download("yolov11")

    # Train model (fewer epochs for comparison)
    %cd {HOME}
    training_output = !yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=20 imgsz=640 plots=True patience=5 name=version_{version_number}

    # Evaluate model
    val_output = !yolo task=detect mode=val model={HOME}/runs/detect/version_{version_number}/weights/best.pt data={dataset.location}/data.yaml

    # Extract metrics
    metrics = {}
    for line in val_output:
        if line.startswith('                   all'):
            parts = line.strip().split()
            metrics = {
                'Precision': float(parts[3]),
                'Recall': float(parts[4]),
                'mAP50': float(parts[5]),
                'mAP50-95': float(parts[6])
            }
            break

    # Calculate F1
    if 'Precision' in metrics and 'Recall' in metrics:
        metrics['F1-Score'] = 2 * (metrics['Precision'] * metrics['Recall']) / (metrics['Precision'] + metrics['Recall'])

    return metrics

In [ ]:
# Compare versions 1 and 2 (version 3 was already evaluated in the main flow)
version1_metrics = train_and_evaluate(1)
version2_metrics = train_and_evaluate(2)

In [ ]:
# Get version 3 metrics from earlier evaluation
version3_metrics = {
    'Precision': overall_metrics['Precision'],
    'Recall': overall_metrics['Recall'],
    'mAP50': overall_metrics['mAP50'],
    'mAP50-95': overall_metrics['mAP50-95'],
    'F1-Score': f1_score
}

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Metric': ['Precision', 'Recall', 'F1-Score', 'mAP50', 'mAP50-95'],
    'Version 1': [version1_metrics.get('Precision', np.nan),
                 version1_metrics.get('Recall', np.nan),
                 version1_metrics.get('F1-Score', np.nan),
                 version1_metrics.get('mAP50', np.nan),
                 version1_metrics.get('mAP50-95', np.nan)],
    'Version 2': [version2_metrics.get('Precision', np.nan),
                 version2_metrics.get('Recall', np.nan),
                 version2_metrics.get('F1-Score', np.nan),
                 version2_metrics.get('mAP50', np.nan),
                 version2_metrics.get('mAP50-95', np.nan)],
    'Version 3': [version3_metrics.get('Precision', np.nan),
                 version3_metrics.get('Recall', np.nan),
                 version3_metrics.get('F1-Score', np.nan),
                 version3_metrics.get('mAP50', np.nan),
                 version3_metrics.get('mAP50-95', np.nan)]
})

In [ ]:
print("\nComparison of Dataset Versions:")
print(comparison_df)

In [ ]:
# Create a visualization of the comparison
plt.figure(figsize=(14, 8))
comparison_melted = pd.melt(comparison_df, id_vars=['Metric'], var_name='Dataset Version', value_name='Value')
sns.barplot(x='Metric', y='Value', hue='Dataset Version', data=comparison_melted)
plt.title('Performance Comparison Across Dataset Versions')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig(f"{HOME}/version_comparison.png", dpi=300, bbox_inches='tight')
plt.show()
```

# Conclusion

In [ ]:
This notebook demonstrates the entire process of training a YOLOv11 model on a biomedical dataset and evaluating its performance using a comprehensive set of metrics including Precision, Recall, F1-score, mAP@50, mAP@50-95, and FPS. These metrics provide a thorough basis for the experimental evaluation section of your research paper.

In [ ]:
The analysis includes:
1. Training a custom YOLOv11 model on biomedical images
2. Evaluating the model with multiple performance metrics
3. Visualizing the results through various plots
4. Comparing performance across different versions of the dataset
5. Generating artifacts suitable for inclusion in a research paper